In [1]:
!pip install --upgrade google-cloud-speech

/anaconda3/lib/python2.7/site-packages/cryptography/hazmat/primitives/constant_time.py:26: CryptographyDeprecationWarning: Support for your Python version is deprecated. The next version of cryptography will remove support. Please upgrade to a 2.7.x release that supports hmac.compare_digest as soon as possible.
  utils.PersistentlyDeprecated2018,
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
     |████████████████████████████████| 92kB 187kB/s eta 0:00:01
     |████████████████████████████████| 71kB 7.6MB/s  eta 0:00:01
     |████████████████████████████████| 81kB 5.1MB/s eta 0:00:011
     |████████████████████████████████| 102kB 296kB/s ta 0:00:01
     |████████████████████████████████| 81kB 336kB/s eta 0:00:011
  Stored in directory: /Users/anshul/Library/Caches/pip/wheels/9e/3d/a2/1bec8bb7db80ab3216dbc33092bb7ccd0debfb

In [9]:
!export GOOGLE_APPLICATION_CREDENTIALS="/Users/anshul/Desktop/ISTEM/azure-speech-testing/My First Project-0c2f8b4259e1.json"

In [3]:
def wer1(ref, hyp ,debug=False):
    ref=ref.lower()
    hyp=hyp.lower()
    r = ref.split()
    h = hyp.split()
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]

    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3

    DEL_PENALTY=1 # Tact
    INS_PENALTY=1 # Tact
    SUB_PENALTY=1 # Tact
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL

    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS

    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1

                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL

    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        print("OP\tREF\tHYP")
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append(".\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("SUB\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("DEL\t" + r[i]+"\t"+"****")
    if debug:
        lines = list(reversed(lines))
        for i,line in enumerate(lines):
            if(line.startswith('.')):
                if(i-1 > 0):
                    if(not lines[i-1].startswith('.')):
                        print(line)
                        continue
                if(i+1 < len(lines)):
                    if(lines[i+1].startswith('.')):continue
            print(line)
        print("Ncor " + str(numCor))
        print("Nsub " + str(numSub))
        print("Ndel " + str(numDel))
        print("Nins " + str(numIns))
    return (numSub + numDel + numIns) / (float) (len(r))
    wer_result = round( (numSub + numDel + numIns) / (float) (len(r)), 3)
    return {'WER':wer_result, 'Cor':numCor, 'Sub':numSub, 'Ins':numIns, 'Del':numDel}

In [7]:
!pip3 install nltk

    100% |████████████████████████████████| 1.5MB 500kB/s ta 0:00:012
  Stored in directory: /Users/anshul/Library/Caches/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
def lemmatized(para):
    para=para.lower()
    def get_wordnet_pos(word):
        """Map POS tag to first character lemmatize() accepts"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

        return tag_dict.get(tag, wordnet.NOUN)

    lemmatizer = WordNetLemmatizer()
    final=''
    sent_text = nltk.sent_tokenize(para) 
    for sentence in sent_text:
        x=([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)])
        x=" ".join(x)+' '
        final=final+x
    return(final)

In [ ]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

# Instantiates a client
client = speech.SpeechClient()

# The name of the audio file to transcribe
file_name = os.path.join(
    os.path.dirname(__file__),
    'resources',
    'audio.raw')

# Loads the audio into memory
with io.open(file_name, 'rb') as audio_file:
    content = audio_file.read()
    audio = types.RecognitionAudio(content=content)

config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code='en-US')

# Detects speech in the audio file
response = client.recognize(config, audio)

for result in response.results:
    print('Transcript: {}'.format(result.alternatives[0].transcript))

In [ ]:
import re
from jiwer import wer
import numpy as np
transcripts=[]
#enter path for audio files to be tested
path='C:\\Users\\eklavya\\istem\\test\\indian\\New Folder\\'
#enter name of audio files to be tested
vids=['Kiran_Bedi_How_I_remade_one_of_Indias_toughest_prisonst0',
      'A_well_educated_mind_vs_a_well_formed_mind_Dr_Shashi_Tharoor_at_TEDxGateway_2013t1',
      "Thoughts_on_humanity_fame_and_love_Shah_Rukh_Khant1",
      "What_makes_life_complete_Gaur_Gopal_Das_TEDxMITPt4",
      "Why_is_India_so_filthy_The_Ugly_Indian_TEDxBangaloret3" 
     ]
for vid in vids:
    print(vid)
    obj1=[]
    speech_recognize_continuous_from_file(path+vid+'.wav')
    st=''
    for i in obj1:
        st=st+' '+((i.result).text)
    transcripts.append(st)

In [ ]:
for i in range(len(vids)):
    print(vids[i],'\t',wers[i])